### Use Scispacy model via UMLS API to get the CUis and extract their LOINC parts

In [ ]:
#ngram approach
import spacy
import scispacy
from scispacy.linking import EntityLinker
import pandas as pd
import requests
import re
from itertools import combinations
import numpy as np

# UMLS API Key — Replace this at runtime - PROVIDE THE API KEY
UMLS_API_KEY = "XXX"  

# Function to clean LOINC part URLs
def clean_lnc_url(lnc_url):
    return lnc_url.replace("https://uts-ws.nlm.nih.gov/rest/content/2024AB/source/LNC/", "")

# Function to fetch LOINC parts from UMLS CUI
def fetch_lnc_from_umls(cui, api_key):
    base_url = f"https://uts-ws.nlm.nih.gov/rest/content/2024AB/CUI/{cui}/atoms?apiKey={api_key}&sabs=LNC"
    try:
        response = requests.get(base_url)
        if response.status_code == 200:
            data = response.json()
            results = data.get("result", [])
            loinc_parts = []

            for item in results:
                loinc_part = item.get("code")
                term_type = item.get("termType")
                if term_type in ["LPDN", "LPN"] and loinc_part:
                    cleaned_part = clean_lnc_url(loinc_part)
                    if cleaned_part not in loinc_parts:
                        loinc_parts.append(cleaned_part)

            return ", ".join(loinc_parts) if loinc_parts else "No Mapping Found"
        return "No Mapping Found"
    except requests.exceptions.RequestException as e:
        print(f"Error fetching LOINC Part for CUI {cui}: {e}")
        return "Error"

# Load models
scibert_nlp = spacy.load("en_core_sci_scibert")
scibert_nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})

scilg_nlp = spacy.load("en_core_sci_lg")
scilg_nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})

print("Models loaded successfully!")

def generate_ngrams(tokens, n):
    return [' '.join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

def extract_entities(doc, nlp_model, row_id, original_text):
    linker = nlp_model.get_pipe("scispacy_linker")
    found = False
    results = []
    for ent in doc.ents:
        for cui, score in ent._.kb_ents:
            concept = linker.kb.cui_to_entity[cui]
            loinc_part = fetch_lnc_from_umls(concept.concept_id, UMLS_API_KEY)
            results.append({
                "Row_ID": row_id,
                "Text": original_text,
                "Start": ent.start_char,
                "End": ent.end_char,
                "Label": ent.label_,
                "UMLS_CUI": concept.concept_id,
                "UMLS_Name": concept.canonical_name,
                "Score": score,
                "LOINC_Part": loinc_part
            })
            found = True
    return results, found

def sliding_window_linking(text, nlp_model, row_id, max_n=3):
    tokens = [token.text for token in nlp_model.tokenizer(text)]
    all_results = []
    found_any = False
    # Generate ngrams from 1 to max_n and run linking
    for n in range(1, min(max_n, len(tokens)) + 1):
        ngrams = generate_ngrams(tokens, n)
        for phrase in ngrams:
            doc = nlp_model(phrase)
            results, found = extract_entities(doc, nlp_model, row_id, text)
            if found:
                all_results.extend(results)
                found_any = True
    return all_results, found_any

# Read input CSV -PROVIDE THE INPUT FILE
input_df = pd.read_csv("filtered_mimic_d_items.csv", keep_default_na=False)

entities = []

for index, row in input_df.iterrows():
    row_id = row['itemid']
    text = row['label'].title()  # Normalize casing here
    
    found_cui = False
    # Try both models in order, stop if found any entities
    for nlp_model in [scilg_nlp, scibert_nlp]:
        results, found = sliding_window_linking(text, nlp_model, row_id, max_n=3)
        if found:
            entities.extend(results)
            found_cui = True
            break

# Convert to DataFrame
entities_df = pd.DataFrame(entities)

#dedup for overlapping n-gram
entities_df = entities_df.drop_duplicates(subset=["Row_ID", "Text", "Start", "End", "UMLS_CUI"])

# Filter rows where LOINC_Part contains LP codes
entities_df = entities_df[entities_df['LOINC_Part'] != "No Mapping Found"]

### pre processing of Loinc parts 

In [ ]:
#loading the file that gives loincpart to unit maps for context ingetsion
units_df = pd.read_csv("loinc_part_unit_prop.csv", keep_default_na=False)

#Explode LOINC_Part column
entities_df = entities_df.assign(LOINC_Part=entities_df['LOINC_Part'].str.split(',')).explode('LOINC_Part')
entities_df['LOINC_Part'] = entities_df['LOINC_Part'].str.strip()

# Load the unwanted UMLS CUIs CSV and strip any leading/trailing spaces
unwanted_umls_df = pd.read_csv('unwanted_system_cuis.csv')

# Remove rows where UMLS_CUI exists in the unwanted CSV (unwanted_umls_df)
# use ~ (negation) to keep only the rows where UMLS_CUI is NOT in the unwanted CSV
entities_df = entities_df[~entities_df['UMLS_CUI'].isin(unwanted_umls_df['UMLS_CUI'])]

# Step 4: Match units from units_df against Text (case-insensitive), add new rows
units_df = units_df.copy()
units_df['units_len'] = units_df['units'].str.len()
units_df = units_df.sort_values('units_len', ascending=False)

new_rows = []
matched_row_ids = set()

for _, unit_row in units_df.iterrows():
    unit = str(unit_row['units']).strip().lower()
    partnumber = unit_row['partnumber']

    if not unit:
        continue

    # Handle '%' as a special case (literal substring match)
    if unit == '%':
        matches = entities_df[entities_df['Text'].str.contains('%', case=False, na=False)]
    else:
        pattern = re.escape(unit)
        matches = entities_df[entities_df['Text'].str.lower().str.contains(pattern, na=False, regex=True)]

    for idx, match_row in matches.iterrows():
        # If this row is already matched by a longer unit, skip it
        if idx in matched_row_ids:
            continue
        matched_row_ids.add(idx)

        new_rows.append({
            "Row_ID": match_row["Row_ID"],
            "Text": match_row["Text"],
            "Start": None,
            "End": None,
            "Label": None,
            "UMLS_CUI": None,
            "UMLS_Name": None,
            "Score": None,
            "LOINC_Part": partnumber
        })
# Append new rows (if any) to entities_df
if new_rows:
    additional_df = pd.DataFrame(new_rows)
    entities_df = pd.concat([entities_df, additional_df], ignore_index=True)

#remove redundant rows
entities_df = entities_df.drop_duplicates(subset=["Row_ID", "Text", "UMLS_CUI", "LOINC_Part"])

### Concerting from LOINC part to LOINC code candidates

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations
from rapidfuzz.fuzz import partial_ratio

def print_size(df, msg):
    print(f"{msg} -> rows: {len(df)}, columns: {len(df.columns)}, memory (MB): {df.memory_usage(deep=True).sum() / (1024*1024):.2f}")

# ---------- Step 1: Read Data ----------
loinc_part = pd.read_csv("LoincPartLink_Primary.csv")
loinc = pd.read_csv('Loinc.csv', dtype=str)
loinc['COMMON_TEST_RANK'] = pd.to_numeric(loinc['COMMON_TEST_RANK'], errors='coerce')

loinc = loinc[['LOINC_NUM', 'COMPONENT', 'PROPERTY', 'SYSTEM', 'METHOD_TYP', 'LONG_COMMON_NAME', 'COMMON_TEST_RANK']]
loinc.rename(columns={'METHOD_TYP': 'METHOD'}, inplace=True)

# Lowercase columns
entities_df.columns = [col.lower() for col in entities_df.columns]
loinc_part.columns = [col.lower() for col in loinc_part.columns]
entities_df = entities_df.rename(columns={'loinc_part': 'partnumber'})

# ---------- Step 2: Merge Entities with LOINC Parts ----------
observation_parts = entities_df.merge(loinc_part, on='partnumber')

# ---------- Step 3: Pivot ----------
pivot = observation_parts.pivot_table(
    index='row_id',
    columns='parttypename',
    values='partname',
    aggfunc=lambda x: list(set(x))
).reset_index()

print_size(pivot, "Pivot after merge")

pivot.columns = [col if isinstance(col, str) else col[1] for col in pivot.columns]
print(pivot.columns.tolist())

# ---------- Step 4: Explode COMPONENT, PROPERTY, SYSTEM, METHOD_TYP ----------
cols_to_explode = ['COMPONENT', 'PROPERTY', 'SYSTEM', 'METHOD']
for col in cols_to_explode:
    if col in pivot.columns:
        pivot[col] = pivot[col].apply(lambda x: x if isinstance(x, list) else [x])
        pivot = pivot.explode(col).reset_index(drop=True)

print_size(pivot, "Pivot after explode")

# ---------- Step 5: Prepare Permutations ----------
# Define all possible axes that could be used
all_possible_axes = ['COMPONENT', 'PROPERTY', 'SYSTEM', 'METHOD']

# Dynamically detect join columns that exist in BOTH pivot and loinc
join_columns = [col for col in all_possible_axes if col in pivot.columns and col in loinc.columns]
print("Join columns actually used:", join_columns)

# Generate all permutations of those columns for matching priority
all_perms = []
for r in range(len(join_columns), 0, -1):
    all_perms.extend(list(combinations(join_columns, r)))

print("All permutations:", all_perms)

# ---------- Step 6: Optimize Data Types ----------
for col in join_columns:
    loinc[col] = pd.Categorical(loinc[col])
    pivot[col] = pd.Categorical(pivot[col])

# ---------- Step 7: Batch Processing ----------
final_results = []

unique_row_ids = pivot['row_id'].unique()
chunk_size = 500  # tune based on memory

for start in range(0, len(unique_row_ids), chunk_size):
    batch_ids = unique_row_ids[start:start + chunk_size]
    batch_df = pivot[pivot['row_id'].isin(batch_ids)]

    batch_output = []

    for rid in batch_df['row_id'].unique():
        row_data = batch_df[batch_df['row_id'] == rid]

        for perm in all_perms:
            perm = list(perm)

            if not set(perm).issubset(set(row_data.columns)):
                continue

            if row_data[perm].isnull().any(axis=None):
                continue

            dedup = row_data.drop_duplicates(subset=['row_id'] + perm)
            joined = dedup.merge(loinc, on=perm)

            if not joined.empty:
                joined['join_strength'] = len(perm)
                joined['join_on'] = ', '.join(perm)
                batch_output.append(joined)
                break  # stop after highest match

    if batch_output:
        batch_result = pd.concat(batch_output, ignore_index=True)
        final_results.append(batch_result)

    print(f"Processed batch {start} to {start + chunk_size}")

# ---------- Step 8: Combine All Batches ----------
if final_results:
    final_df = pd.concat(final_results, ignore_index=True)
else:
    final_df = pd.DataFrame()

print_size(final_df, "Final joined dataframe")

# ---------- NEW STEP 8.5: Compute Robust Composite Score ----------

# Get max join_strength from final_df (handle case if empty)
max_join_strength = final_df['join_strength'].max() if not final_df.empty else 1

# Precompute row_id → input text map
row_id_to_text = (
    entities_df.dropna(subset=['text'])
    .groupby('row_id')['text']
    .first()
    .str.lower()
    .to_dict()
)

def input_aware_axis_score(row):
    # Normalize join_strength to [0,1]
    axis_match_score = row['join_strength'] / max_join_strength if max_join_strength > 0 else 0

    rank = row['COMMON_TEST_RANK']
    if pd.isna(rank) or rank <= 0:
        rank_score = 0
    else:
        max_rank = 30000
        rank_score = 1 - (rank / max_rank)
        rank_score = max(rank_score, 0)

    # Fuzzy similarity score (input text vs LOINC name)
    input_text = row_id_to_text.get(row['row_id'], '').lower()
    loinc_name = str(row.get('LONG_COMMON_NAME', '')).lower()
    if input_text and loinc_name:
        sim_score = partial_ratio(input_text, loinc_name) / 100  # Normalize to 0–1
    else:
        sim_score = 0

    # Weights
    w_axis = 0.5
    w_rank = 0.3
    w_sim = 0.3

    return w_axis * axis_match_score  + w_rank * rank_score + w_sim * sim_score


# Apply composite score
final_df['composite_score'] = final_df.apply(input_aware_axis_score, axis=1)

# Sort by composite score
#final_df = final_df.sort_values(by=['row_id', 'composite_score'], ascending=[True, False])

print_size(final_df, "Final dataframe after composite scoring")
#---end of new section


# ---------- Step 9: Sorting + Top 10 ----------
final_df['sort_rank'] = pd.to_numeric(final_df['COMMON_TEST_RANK'], errors='coerce').replace(0, np.nan)

final_df = (
    final_df.sort_values(by=['row_id', 'join_strength', 'sort_rank'], ascending=[True, False, True])
    .groupby('row_id')
    .head(10)
    .reset_index(drop=True)
)

final_df = final_df.drop(columns=['sort_rank'])

print_size(final_df, "Final dataframe after sorting")

# ----------  Done  ----------

# Apply hardcoded LOINC mappings to existing rows OF HEMOGRAM RELATED STRINGS

# Define hardcoded mappings
hardcoded_mappings = {
    'mch': {
        'LOINC_NUM': '28539-5',
        'LONG_COMMON_NAME': 'MCH [Entitic mass]',
        'COMMON_TEST_RANK': 104,
        'join_strength': 1,
        'join_on': 'BROAD',
        'composite_score': '0.9'
    },
    'mcv': {
        'LOINC_NUM': '787-2',
        'LONG_COMMON_NAME': 'MCV [Entitic mean volume] in Red Blood Cells by Automated count',
        'COMMON_TEST_RANK': 14,
        'join_strength': 1,
        'join_on': 'BROAD',
        'composite_score': '0.9'
    },
    'mchc': {
        'LOINC_NUM': '786-4',
        'LONG_COMMON_NAME': 'MCHC [Entitic Mass/volume] in Red Blood Cells by Automated count',
        'COMMON_TEST_RANK': 13,
        'join_strength': 1,
        'join_on': 'BROAD',
        'composite_score': '0.9'
    },
    'hct': {
        'LOINC_NUM': '4544-3',
        'LONG_COMMON_NAME': 'Hematocrit [Volume Fraction] of Blood by Automated count',
        'COMMON_TEST_RANK': 28,
        'join_strength': 1,
        'join_on': 'BROAD',
        'composite_score': '0.9'
    },
    'heart rate': {
        'LOINC_NUM': '8867-4',
        'LONG_COMMON_NAME': 'Heart rate',
        'COMMON_TEST_RANK': 18,
        'join_strength': 1,
        'join_on': 'BROAD',
        'composite_score': '0.9'
    },
    'fetal heart rate': {
        'LOINC_NUM': '55283-6',
        'LONG_COMMON_NAME': 'Fetal Heart rate',
        'COMMON_TEST_RANK': 2568,
        'join_strength': 1,
        'join_on': 'BROAD',
        'composite_score': '0.9'
    },
    'weight': {
        'LOINC_NUM': '29463-7',
        'LONG_COMMON_NAME': 'Body weight',
        'COMMON_TEST_RANK': 44,
        'join_strength': 1,
        'join_on': 'BROAD',
        'composite_score': '0.9'
    }
}

# Iterate through each keyword and update matching rows
for keyword, values in hardcoded_mappings.items():
    mask = entities_df['text'].str.lower().str.contains(keyword, na=False)
    matching_row_ids = entities_df.loc[mask, 'row_id'].tolist()
    
    if matching_row_ids:
        update_mask = final_df['row_id'].isin(matching_row_ids)
        
        final_df.loc[update_mask, 'LOINC_NUM'] = values['LOINC_NUM']
        final_df.loc[update_mask, 'LONG_COMMON_NAME'] = values['LONG_COMMON_NAME']
        final_df.loc[update_mask, 'COMMON_TEST_RANK'] = values['COMMON_TEST_RANK']
        final_df.loc[update_mask, 'join_strength'] = values['join_strength']
        final_df.loc[update_mask, 'join_on'] = values['join_on']
        final_df.loc[update_mask, 'composite_score'] = values['composite_score']

print_size(final_df, "Final dataframe after hardcoded updates")

final_df = final_df.drop_duplicates(
    subset=['row_id', 'LOINC_NUM', 'join_on']
).reset_index(drop=True)

### final get the output

In [ ]:
#Final view - Deduplicate to keep one text per row_id
entity_texts = entities_df[['row_id', 'text']].drop_duplicates()

#Merge with top 15 LOINC matches from final_df
entities_top15 = entity_texts.merge(
    final_df[['row_id', 'LOINC_NUM', 'LONG_COMMON_NAME', 'COMMON_TEST_RANK', 'join_strength', 'join_on', 'composite_score']],
    how='left',
    on='row_id'
)

#Reset index (optional for cleanliness)
entities_top15 = entities_top15.reset_index(drop=True)

#optional export if needed
merged_df.to_csv("mapping file.csv", index=False)